In [1]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A100-PCIE-40GB, 40960 MiB, 28211 MiB
NVIDIA A100-PCIE-40GB, 40960 MiB, 38895 MiB
NVIDIA A100-PCIE-40GB, 40960 MiB, 40355 MiB


In [2]:
import torch
import os
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

SEED = None
DEVICE = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

pretrained_model_name_or_path = "/root/autodl-tmp/stable_diffusion/stable-diffusion-2"
original_path = "/root/autodl-tmp/textual_inversion/original"

tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer",
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder", torch_dtype=torch.float16
)

scheduler = EulerDiscreteScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_name_or_path,
                                               scheduler=scheduler,
                                               torch_dtype=torch.float16, 
                                               text_encoder=text_encoder, 
                                               tokenizer=tokenizer).to(DEVICE)

/root/miniconda3/envs/dushian/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/dushian/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [25]:
prompt = "a small siberian kitten with blue eyes and brown fur"
generator = None if SEED is None else torch.Generator(
            device=DEVICE).manual_seed(SEED)

image = pipe(prompt, generator=generator).images[0]

image_dir = os.path.join(original_path, "images_v2")
os.makedirs(image_dir, exist_ok=True)
image_path = os.path.join(image_dir, "{}.png".format(prompt))
image.save(image_path)

100%|██████████| 50/50 [00:04<00:00, 11.24it/s]
